In [15]:
import pandas as pd
import numpy as np
import torch
from models.agent import DQNAgent
from strategies.trading_env import TradingEnvironment
import json
import os


In [16]:
ACTIONS = {0: "Hold", 1: "Enter", 2: "Exit"}
ACTION_SPACE = list(ACTIONS.keys())

# Charger les données complètes et grouper par paire
full_data = pd.read_csv(r"C:\Users\lbenzemma\Desktop\Projets Master2 MOSEF\Kalman-Filtering-Applied-to-Investment-Portfolio-Management-1\data\cointegrated_pairs_spreads.csv")

# Normaliser les spreads dans le dataset principal
full_data['spread'] = full_data.groupby(['Asset1', 'Asset2'])['Spread'].transform(lambda x: (x - x.mean()) / x.std())

rewards_by_pair = {}
output_dir = "saved_models"
os.makedirs(output_dir, exist_ok=True)


In [17]:
def train_on_pair(df_pair, pair_id, num_episodes=100):
    env = TradingEnvironment(df_pair)
    agent = DQNAgent(input_dim=3, output_dim=3)

    rewards_all = []
    best_reward = -np.inf
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False
        epsilon = max(0.1, 1.0 - episode / 50)

        while not done:
            action = agent.get_action(state, epsilon)
            next_state, reward, done = env.step(action)
            agent.train_step(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

        rewards_all.append(total_reward)
        if total_reward > best_reward:
            best_reward = total_reward
            agent.save(f"{output_dir}/{pair_id}.pt")

        if episode % 10 == 0:
            print(f"Episode {episode}: Total reward = {total_reward:.2f}")
    return rewards_all

In [18]:
best_5_pairs = (
    full_data[['Asset1', 'Asset2', 'P-value']]
    .drop_duplicates()
    .sort_values(by='P-value')
    .head(5)
    .reset_index(drop=True)
)

print(" 5 meilleures paires co-intégrées sélectionnées :")
print(best_5_pairs)


 5 meilleures paires co-intégrées sélectionnées :
   Asset1  Asset2   P-value
0  IMP.JO  AEG.JO  0.000003
1  AMS.JO  MUR.JO  0.005944
2  FSR.JO  DSY.JO  0.008962
3  AMS.JO  AEG.JO  0.011692
4  IMP.JO  MUR.JO  0.019504


In [ ]:
for _, row in best_5_pairs.iterrows():
    pair_id = f"{row['Asset1']}-{row['Asset2']}"
    df_pair = full_data[(full_data['Asset1'] == row['Asset1']) & (full_data['Asset2'] == row['Asset2'])].copy()
    try:
        print(f"\n==== Training on best pair: {pair_id} ====")
        rewards = train_on_pair(df_pair, pair_id, num_episodes=100)
        rewards_by_pair[pair_id] = rewards
    except Exception as e:
        print(f" Skipping pair {pair_id} due to error: {e}")

# Sauvegarder les résultats dans un fichier JSON
with open("rewards_by_pair.json", "w") as f:
    json.dump(rewards_by_pair, f, indent=2)
print("\n Résultats enregistrés dans rewards_by_pair.json")
